In [2]:
! pip install beautifulsoup4

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [3]:
! pip install requests

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [4]:
! pip install lxml

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [5]:
! pip install folium

    100% |████████████████████████████████| 92kB 15.1MB/s ta 0:00:01
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [6]:
#import liabriaries

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# PART 1

**In the following code, we will read the html data from Wiki and clean the data.  The data can be found at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M** 

In [7]:
#import libraries
import pandas as pd
import numpy as np

In [8]:
#use pandas to read in the html info
# create a dataframe from the first table
data_toronto_df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header = 0)[0]

In [9]:
data_toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [10]:
# preserve the order of the columns
columns = list(data_toronto_df.columns)

#reset the index to Borough
data_toronto_df = data_toronto_df.set_index('Borough')

#delete all rows whose Borough = Not Assigned
data_toronto_df = data_toronto_df.drop("Not assigned", axis = 0)

# reinstate Borough as column
data_toronto_df = data_toronto_df.reset_index()


In [11]:
data_toronto_df.head()

,Borough,Postcode,Neighbourhood
0,North York,M3A,Parkwoods
1,North York,M4A,Victoria Village
2,Downtown Toronto,M5A,Harbourfront
3,Downtown Toronto,M5A,Regent Park
4,North York,M6A,Lawrence Heights


In [12]:
#reinstate the orgignal order of the columns
data_toronto_df = data_toronto_df[columns]
data_toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [13]:
#define a function to deal with the "Not assigned" Neighbourhoods 

def fun_replace(row):
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
    
    return row['Neighbourhood']

#replace the values of Not Assigned in the Neighbourhoods with the values of the Boroughs    
data_toronto_df['Neighbourhood'] = data_toronto_df.apply(fun_replace, axis = 1) 

In [14]:
#check there are no "Not assigned" neighborhoods
(data_toronto_df['Neighbourhood'] != 'Not assigned').all()

True

In [15]:
data_toronto_df.head(3)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront


In [16]:
data_toronto_grouped_df = data_toronto_df.groupby('Postcode', as_index = False).agg(lambda x: ', '.join(set(x)))

In [17]:
print("The number of rows in the cleaned datafrane is ", data_toronto_grouped_df.shape[0])

The number of rows in the cleaned datafrane is  103


# PART 2

In [18]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client  #private data removed 
        # add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()



,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df_data_1.shape

(103, 3)

In [20]:
df_data_1 = df_data_1.rename(columns = {'Postal Code' : 'PostalCode'})
data_toronto_grouped_df = data_toronto_grouped_df.rename(columns = {'Postcode' : 'PostalCode'})

In [21]:
data_toronto_grouped_df = data_toronto_grouped_df.merge(df_data_1, on = 'PostalCode')

In [22]:
data_toronto_grouped_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, West Hill, Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
